# **BERTopic - Tutorial**

In [ ]:
!pip install bertopic[visualization] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 55.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.2 MB/s eta 0:00:00


# **Imports**

In [ ]:
import numpy as np
import pandas as pd
from copy import deepcopy
from bertopic import BERTopic

In [ ]:
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# **Load data**

In [ ]:
df = pd.read_excel("/content/data_positif.xlsx")

In [ ]:
df.head()

,stemming_content,label
0,elegan nyaman aplikasi enteng mudah paham,positif
1,terima kasih tolong batasin konten negatif bia...,positif
2,tolong mentraslite postingan akun,positif
3,haloo erortapi notif muncul,positif
4,lancar samsung galaxy chipset hio fitur rilis ...,positif


In [ ]:
df=df.dropna(subset=['label'])

In [ ]:
df =df.drop(["label"], axis = 1)

In [ ]:
df.head()

,stemming_content
0,elegan nyaman aplikasi enteng mudah paham
1,terima kasih tolong batasin konten negatif bia...
2,tolong mentraslite postingan akun
3,haloo erortapi notif muncul
4,lancar samsung galaxy chipset hio fitur rilis ...


In [ ]:
df['stemming_content']=df['stemming_content'].str.lower()
df.head(150)

,stemming_content
0,elegan nyaman aplikasi enteng mudah paham
1,terima kasih tolong batasin konten negatif bia...
2,tolong mentraslite postingan akun
3,haloo erortapi notif muncul
4,lancar samsung galaxy chipset hio fitur rilis ...
...,...
145,orang tolong
146,bintang nyimpan foto tolong kasih vitur nyimpa...
147,komen negatif sabar app proses kembang
148,nge bug tolong


In [ ]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [ ]:
df["stemming_content"] = df["stemming_content"].apply(lambda text: remove_punctuation(text))
df.head()

,stemming_content
0,elegan nyaman aplikasi enteng mudah paham
1,terima kasih tolong batasin konten negatif bia...
2,tolong mentraslite postingan akun
3,haloo erortapi notif muncul
4,lancar samsung galaxy chipset hio fitur rilis ...


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'ada, adalah, adanya, adapun, agak, agaknya, agar, akan, akankah, akhir, akhiri, akhirnya, aku, akulah, amat, amatlah, anda, andalah, antar, antara, antaranya, apa, apaan, apabila, apakah, apalagi, apatah, artinya, asal, asalkan, atas, atau, ataukah, ataupun, awal, awalnya, bagai, bagaikan, bagaimana, bagaimanakah, bagaimanapun, bagi, bagian, bahkan, bahwa, bahwasanya, baik, bakal, bakalan, balik, banyak, bapak, baru, bawah, beberapa, begini, beginian, beginikah, beginilah, begitu, begitukah, begitulah, begitupun, bekerja, belakang, belakangan, belum, belumlah, benar, benarkah, benarlah, berada, berakhir, berakhirlah, berakhirnya, berapa, berapakah, berapalah, berapapun, berarti, berawal, berbagai, berdatangan, beri, berikan, berikut, berikutnya, berjumlah, berkali-kali, berkata, berkehendak, berkeinginan, berkenaan, berlainan, berlalu, berlangsung, berlebihan, bermacam, bermacam-macam, bermaksud, bermula, bersama, bersama-sama, bersiap, bersiap-siap, bertanya, bertanya-tanya, berturut

In [ ]:
STOPWORDS = set(stopwords.words('indonesian'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [ ]:
df["stemming_content"] = df["stemming_content"].apply(lambda text: remove_stopwords(text))
df.head(50)

,stemming_content
0,elegan nyaman aplikasi enteng mudah paham
1,terima kasih tolong batasin konten negatif bia...
2,tolong mentraslite postingan akun
3,haloo erortapi notif muncul
4,lancar samsung galaxy chipset hio fitur rilis ...
5,ganti twitter lihat iya btw scroll asa smooth
6,bingung kerja wkkw
7,tolong fitur chat kirim pesan please chat send...
8,fitur moga ya
9,overall oke masuk tolong tambahakan fitur tren...


In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
df["stemming_content"] = df["stemming_content"].apply(lambda text: remove_emoji(text))
df.head(200)

,stemming_content
0,elegan nyaman aplikasi enteng mudah paham
1,terima kasih tolong batasin konten negatif bia...
2,tolong mentraslite postingan akun
3,haloo erortapi notif muncul
4,lancar samsung galaxy chipset hio fitur rilis ...
...,...
195,mudah operasi
196,kembang download buka tunggu
197,malaz support
198,trending hastag seru twitter


In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


In [ ]:
df["stemming_content"] = df["stemming_content"].apply(lambda text: lemmatize_words(text))
df.head(200)

,stemming_content
0,elegan nyaman aplikasi enteng mudah paham
1,terima kasih tolong batasin konten negatif bia...
2,tolong mentraslite postingan akun
3,haloo erortapi notif muncul
4,lancar samsung galaxy chipset hio fitur rilis ...
...,...
195,mudah operasi
196,kembang download buka tunggu
197,malaz support
198,trending hastag seru twitter


In [ ]:
from collections import Counter
cnt = Counter()
for text in df["stemming_content"].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('fitur', 106),
 ('tolong', 102),
 ('foto', 83),
 ('thread', 74),
 ('bug', 54),
 ('moga', 49),
 ('bagus', 39),
 ('kasih', 38),
 ('instagram', 35),
 ('twitter', 32)]

In [ ]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["stemming_content"] = df["stemming_content"].apply(lambda text: remove_freqwords(text))
df.head(10)

,stemming_content
0,elegan nyaman aplikasi enteng mudah paham
1,terima batasin konten negatif biar kayak komun...
2,mentraslite postingan akun
3,haloo erortapi notif muncul
4,lancar samsung galaxy chipset hio rilis update...
5,ganti lihat iya btw scroll asa smooth
6,bingung kerja wkkw
7,chat kirim pesan please chat send message feature
8,ya
9,overall oke masuk tambahakan trending tarik th...


In [ ]:
docs = list(df.loc[:, "stemming_content"].values)

# **Creating Topics**

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
sentence_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=True)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
model = BERTopic("indolem/indobert-base-uncased",  calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform (docs ,embeddings)

# We can then extract most frequent topics:

In [ ]:
model.get_topic_freq()

,Topic,Count
1,-1,108
0,0,51
6,1,46
10,2,37
4,3,26
7,4,25
2,5,22
5,6,20
8,7,17
13,8,17


# Get Individual Topics

In [ ]:
model.get_topic(-1)

[('chat', 0.04995302879554217),
 ('enak', 0.04553695161491683),
 ('post', 0.04553695161491683),
 ('close', 0.04553695161491683),
 ('force', 0.043708900196099396),
 ('you', 0.03746477159665663),
 ('edit', 0.029375768273599823),
 ('mode', 0.029375768273599823),
 ('pakai', 0.02887430665119407),
 ('banget', 0.028622034365899645)]

In [ ]:
model.get_topic(0)

[('aplikasi', 0.10813368885471396),
 ('mudah', 0.047915991352677066),
 ('saran', 0.04677436877836566),
 ('terima', 0.04677436877836566),
 ('tread', 0.04510540963596315),
 ('terimakasih', 0.04325145612236611),
 ('keren', 0.040124185596773776),
 ('suka', 0.03531419359893721),
 ('kayak', 0.0342069653708604),
 ('lencana', 0.03382905722697236)]

In [ ]:
model.get_topic(1)

[('upload', 0.138168873201318),
 ('video', 0.1269126501645507),
 ('vidio', 0.07188470821763526),
 ('save', 0.07082258498946435),
 ('kualitas', 0.05379864538479037),
 ('postingan', 0.041698080693891866),
 ('masuk', 0.0361892240102495),
 ('suara', 0.0361892240102495),
 ('kali', 0.035411292494732176),
 ('orang', 0.0353811099849503)]

In [ ]:
model.get_topic(2)

[('apk', 0.16669201842675932),
 ('android', 0.07113422100858699),
 ('developer', 0.06439671777829825),
 ('banget', 0.060106272168389256),
 ('app', 0.055514578338095755),
 ('bikin', 0.05088222990352509),
 ('seru', 0.03704979469049263),
 ('good', 0.03704979469049263),
 ('semangat', 0.03592332604487928),
 ('this', 0.03592332604487928)]

In [ ]:
model.get_topic(3)

[('hastag', 0.2614380156570072),
 ('trending', 0.18190032302697756),
 ('search', 0.10524595581724736),
 ('trend', 0.08504397837887273),
 ('kolom', 0.07422701322635165),
 ('topic', 0.06247534964326832),
 ('bugnya', 0.06247534964326832),
 ('bookmark', 0.06247534964326832),
 ('uta', 0.0598621893990057),
 ('cari', 0.05669598558591516)]

In [ ]:
model.get_topic(4)

[('translate', 0.41625092656823615),
 ('erti', 0.12485179804557911),
 ('bahasa', 0.12485179804557911),
 ('adakan', 0.09104142861425481),
 ('terima', 0.08011354652464756),
 ('artis', 0.07725501012117092),
 ('terimakasih', 0.07407962165639301),
 ('bingung', 0.07264771507259948),
 ('suka', 0.060484948610945645),
 ('bintang', 0.056695739579331834)]

In [ ]:
model.get_topic(5)

[('terjemah', 0.4644747448264505),
 ('otomatis', 0.33308747002857453),
 ('vitur', 0.1156670875772993),
 ('terima', 0.08965087349186751),
 ('tulis', 0.08965087349186751),
 ('erortapi', 0.08923593346007347),
 ('macet', 0.08923593346007347),
 ('nyatu', 0.08923593346007347),
 ('haloo', 0.08923593346007347),
 ('akurat', 0.08923593346007347)]

In [ ]:
model.get_topic(6)

[('download', 0.27609511875356085),
 ('coba', 0.14507259241043632),
 ('install', 0.12225071891962956),
 ('nyimpan', 0.1134682961154698),
 ('langsung', 0.10227133990389137),
 ('instal', 0.0898083151121982),
 ('buka', 0.06371425817389867),
 ('saran', 0.05229634287025604),
 ('lebi', 0.05205429451837619),
 ('dempet', 0.05205429451837619)]

In [ ]:
model.get_topic(7)

[('follow', 0.255864927514869),
 ('follower', 0.2060694968905544),
 ('timeline', 0.15422278343639906),
 ('orang', 0.13523446483136564),
 ('following', 0.13007400412998768),
 ('lihat', 0.11220627006598367),
 ('refresh', 0.06916162810847684),
 ('stuff', 0.047592497845372526),
 ('annoying', 0.047592497845372526),
 ('another', 0.047592497845372526)]

In [ ]:
model.get_topic(8)

[('suka', 0.1439388650488327),
 ('ngebug', 0.11738135898829048),
 ('tulis', 0.09532497940907432),
 ('saing', 0.08644158502309306),
 ('gambar', 0.07825423932552698),
 ('buka', 0.07742492132524395),
 ('akun', 0.06740325383208888),
 ('orang', 0.06740325383208888),
 ('aduk', 0.06325585156663437),
 ('maahh', 0.06325585156663437)]

In [ ]:
model.get_topic(9)

[('mantap', 0.543999484722575),
 ('manfaat', 0.3119982710778327),
 ('kembang', 0.18785890532606636),
 ('saing', 0.179707505705904),
 ('nyaman', 0.14024735580150505),
 ('cakep', 0.13150558615168723),
 ('mwehehe', 0.13150558615168723),
 ('sepi', 0.13150558615168723),
 ('game', 0.13150558615168723),
 ('ui', 0.13150558615168723)]

In [ ]:
model.get_topic(10)

[('update', 0.2632244975694515),
 ('tunggu', 0.09166959511049724),
 ('diupdate', 0.08888245619351574),
 ('kutip', 0.08066026815315765),
 ('sayang', 0.0748656799112378),
 ('refresh', 0.0748656799112378),
 ('cepat', 0.07040087852396239),
 ('deh', 0.05677650953307757),
 ('nyaman', 0.05494226309749683),
 ('akun', 0.05489543353335074)]

In [ ]:
model.get_representative_docs(-1)

['uta force close',
 'salah tema chat menginstall tema chat default sisi ubah default cuman lihat chat tema algoritmanya arah artis isi artis centang biru send chat model cuman nampil',
 'post force close']

In [ ]:
model.get_representative_docs(0)

['aplikasi bawaanya positif sharing ilmu saran tingkat aplikasi terimakasih',
 'aplikasi rilis tahap kembang bilang temu aplikasi aplikasi rilis beres tol pemograman mudah itung duit',
 'aplikasi tread mudah paham simple kayak alam sosial medium']

In [ ]:
model.get_representative_docs(1)

['upload', 'video upload suara', 'upload video']

In [ ]:
model.get_representative_docs(2)

['banget seru dukung apk',
 'banget ketemu konten nsfw size apk berat buffering konten banget beda kayak simple saran size audio meetingnya mudah bantu developer',
 'apk saing apk translate otomatis']

In [ ]:
model.get_representative_docs(3)

['hastag', 'hastag trending terjemah', 'hastag']

In [ ]:
model.get_representative_docs(4)

['translate', 'translate', 'translate']

In [ ]:
model.get_representative_docs(5)

['terjemah otomatis terima', 'terjemah otomatis', 'terjemah otomatis']

In [ ]:
model.get_representative_docs(6)

['mohon download', 'saran download adakan biar enak nyimpan', 'download orang']

In [ ]:
model.get_representative_docs(7)

['lihat follower following',
 'rapihkan notifnya follback update follower profil follower lihat stuck situ lihat follower following orang',
 'aneh lihat orang follow timeline orang follow follow orang']

In [ ]:
model.get_representative_docs(8)

['suka ngebug log tahap kembang',
 'buka ngebug gambar tulis campur aduk buka ulang kali',
 'suka nama orang suka uta cuman tulis lihat suka']

In [ ]:
model.get_representative_docs(9)

['mantap saing wkwk', 'mantap kembang sukses', 'mantap min kembang manfaat']

# **Visualize Topics**

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_hierarchy()